## O que é Delta Lake?

https://learn.microsoft.com/pt-br/azure/databricks/delta/

Delta Lake é uma tecnologia de armazenamento open source que traz confiabilidade, desempenho e governança para os data lakes. Ela permite a criação de pipelines de dados robustos, garantindo transações ACID, versionamento de dados e suporte a operações de leitura e escrita simultâneas. Com Delta Lake, é possível unificar dados estruturados e semiestruturados em um único local, facilitando análises avançadas e machine learning em larga escala.

* **Delta Lake**: É a camada de armazenamento otimizada que fornece a base para armazenar dados e tabelas na Plataforma Databricks Lakehouse

* **Lakehouse**: É o ambiente onde fica armazenado arquivos\tabelas no formato Delta

* **Diferença básica de Delta lake e Data lake**: O que acontece quando apaga dados dos 2 ambientes? voce sabe? Se eu apagar algo no meus dado no ambiente de Data Lake eu teria que refazer a ingestão da minha base novamente. Já no Delta Lake temos versionamento, é só voltar a versão anterior dos dados.

* **Transações ACID**: significa atomicidade, consistência, isolamento e durabilidade.

  * **Atomicidade**: significa que todas as transações têm êxito ou falham completamente.
  * **Consistência**: estão relacionadas a como um determinado estado dos dados é observado por operações simultâneas.
  * **Isolamento**: se refere a como as operações simultâneas podem entrar em conflito entre si.
  * **Durabilidade**: significa que as alterações confirmadas são permanentes.

* **Benefícios**:

  * **Armazenamento otimizado**: O Delta Lake é o software de código aberto que estende arquivos de dados Parquet com log e extramamente eficiente e compactado

  * Segurança e controle do fluxo de dados e recuperação se nescessário 

  * **Versionamento dos dados**:  Cada gravação em uma tabela Delta cria uma versão da tabela. Você pode usar o log de transações para examinar modificações em sua tabela e consultar versões anteriores Você pode recuperar informações de operações, usuário, carimbo de data/hora e assim por diante relativas a cada gravação em uma tabela do Delta por meio do comando history. As operações são retornadas em ordem cronológica inversa. A retenção do histórico de tabelas é determinada pela configuração da tabela `delta.logRetentionDuration`, que é de 30 dias por padrão.

* **Exemplo de data lake**: https://www.databricks.com/br/glossary/data-lakehouse

Referencia ACID

https://learn.microsoft.com/pt-br/azure/databricks/lakehouse/acid 

Referencia SQL

https://learn.microsoft.com/pt-br/azure/databricks/sql/language-manual/

https://learn.microsoft.com/pt-br/azure/databricks/delta/tutorial

Referencia Dataflame

https://docs.delta.io/latest/api/python/index.html

Lakehouse

https://www.databricks.com/br/blog/2020/01/30/what-is-a-data-lakehouse.html


In [0]:
# Salvando em tabela Delta simples para verificarmos os logs de transações 
spark.read.json("/Volumes/workspace/default/tutorial/anac/V_OCORRENCIA_AMPLA.json") \
    .write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("anac")

In [0]:
%sql
select * from anac

In [0]:
%sql 
delete from anac
where Aerodromo_de_Origem is null;

In [0]:
%sql 
delete from anac
where UF = "MG";

In [0]:
%sql 
-- testando os deletes
select * from anac
where UF = "SP";

###Versionamento, verificando e recuperando dados 

O Delta Lake oferece recursos avançados de versionamento de dados, permitindo rastrear todas as alterações realizadas em uma tabela ao longo do tempo. Com o versionamento, é possível consultar versões anteriores dos dados, auditar modificações e até mesmo restaurar registros excluídos ou alterados acidentalmente. Esses recursos são fundamentais para garantir governança, rastreabilidade e segurança em ambientes de dados modernos.

A retenção do histórico de tabelas é determinada pela configuração da tabela `delta.logRetentionDuration`, que é de 30 dias por padrão
https://learn.microsoft.com/pt-br/azure/databricks/delta/history

In [0]:
%sql
-- Ver logs transacionais
DESCRIBE HISTORY anac 

In [0]:
%sql
-- Ver detalhes da estrutura Delta
DESCRIBE DETAIL anac

In [0]:
%sql 
-- Vendo versao de log na prática 
select * from anac VERSION AS OF 1


In [0]:
%sql
--- Restaurando uma versao (fazer consulta antes de restaurar para ver funcionando na prática)
select * from anac where UF = "MG";

In [0]:
%sql
RESTORE TABLE anac TO VERSION AS OF 1

In [0]:
%sql
select * from anac where UF = "MG";

### Salvando de modo particionado

O particionamento é uma técnica fundamental no armazenamento de dados em larga escala, permitindo dividir grandes volumes de informações em partes menores e organizadas com base em uma ou mais colunas. Ao salvar dados de modo particionado no Delta Lake, é possível otimizar consultas, acelerar o processamento e reduzir custos de leitura, já que apenas as partições relevantes são acessadas durante as operações. Essa abordagem é especialmente útil em cenários de big data, onde a eficiência e a escalabilidade são essenciais para o desempenho das aplicações analíticas.

In [0]:
# Salvando em modo particionado 
# Obs: particionar o que realmente for nescessárrio para otimizar a consulta (lembra das aulas de particionamento)
spark.read.json("/Volumes/workspace/default/tutorial/anac/V_OCORRENCIA_AMPLA.json") \
    .write.format("delta") \
    .partitionBy("UF") \
    .mode("overwrite") \
    .saveAsTable("anac_particionado")

In [0]:
%sql 
select * from anac_particionado
where UF = "MG"

In [0]:
%sql 
delete from anac_particionado
where Aerodromo_de_Destino = 'SBMK' 

In [0]:
%sql 
describe history anac_particionado;

### Otimizando Consulta 

https://learn.microsoft.com/pt-br/azure/databricks/delta/tutorial

https://learn.microsoft.com/pt-br/azure/databricks/delta/optimize

https://learn.microsoft.com/pt-br/azure/databricks/delta/vacuum

A otimização de consultas em tabelas Delta Lake é fundamental para garantir alta performance e eficiência no processamento de grandes volumes de dados. O Delta Lake oferece comandos e práticas recomendadas, como o uso do `OPTIMIZE` para compactação de arquivos e o `VACUUM` para limpeza de arquivos obsoletos, que ajudam a melhorar a velocidade das consultas e a organização do armazenamento. Nesta seção, veremos como aplicar essas técnicas para obter o máximo desempenho em ambientes de Lakehouse.

In [0]:
spark.read.json("/Volumes/workspace/default/tutorial/anac/V_OCORRENCIA_AMPLA.json") \
    .write.mode("overwrite") \
    .saveAsTable("anac_normal")

In [0]:
%sql
select
  replace(Aerodromo_de_Origem,"Sem Origem") as Origem,
  Fase_da_Operacao,
  Danos_a_Aeronave,
  Municipio,
  UF as Estado,
  Operacao,
  Regiao,
  count(Matricula)  as quantidade
from anac_normal
group by 1,2,3,4,5,6,7
having Origem is not null
and Danos_a_Aeronave = 'Nenhum'
order by 1,2;

#### Formato Otimizado

O Delta Lake utiliza um formato de armazenamento otimizado baseado em arquivos Parquet, enriquecido com um log de transações que garante confiabilidade, desempenho e governança dos dados. Esse formato permite operações ACID, versionamento e gerenciamento eficiente de grandes volumes de dados, tornando o Delta Lake ideal para ambientes de lakehouse e análises avançadas.

```sql
OPTIMIZE nome_da_tabela  
ZORDER BY (nome_coluna_ordenada)
```

**Obs**: cria tipo um indice nas colunas pode ser mais de 1, se der erro (`IllegalArgumentException: Danos_a_Aeronave is a partition column. Z-Ordering can only be performed on data columns`) quer dizer que a otimização Z-Ordering que você está tentando aplicar não será eficaz porque as estatísticas não estão sendo coletadas para as colunas especificadas ou então esta tentando fazer pela partição (no nosso exemplo foi criado a partição Danos_a_Aeronave nao posso fazer ZORDER BY por ela).


```sql
set spark.databricks.delta.optimize.zorder.checkStatsCollection.enabled = False;
```

In [0]:
# Salvando no formato delta 
spark.read.json("/Volumes/workspace/default/tutorial/anac/V_OCORRENCIA_AMPLA.json") \
    .write.format("delta") \
    .partitionBy("Danos_a_Aeronave") \
    .mode("overwrite") \
    .saveAsTable("anac_delta_novo")

In [0]:
%sql
-- Otimizar a tabela anac_delta_novo pela coluna Aerodromo_de_Origem
-- O ZORDER BY vai criar um indice da coluna
-- Não pode fazer otimização de coluna particionada, vai dar error.
OPTIMIZE anac_delta_novo  
ZORDER BY (Aerodromo_de_Origem);

In [0]:
%sql
-- A cada execução ele vai diminuindo o tempo.
select
  replace(Aerodromo_de_Origem, "Sem Origem") as Origem,
  Fase_da_Operacao,
  Danos_a_Aeronave,
  Municipio,
  UF as Estado,
  Operacao,
  Regiao,
  count(Matricula) as quantidade
from anac_delta_novo
group by 1,2,3,4,5,6,7
having Origem is not null
and Danos_a_Aeronave = 'Nenhum'
order by 1,2;